Import data from Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/AB_job_data_files'

import os
from google.colab import drive

# find location of different files:
file_location = {}
# Check if the folder exists
if os.path.exists(folder_path):
  os.chdir(folder_path)
  for num, f in enumerate(os.listdir()):  # List files and directories in the current folder
    file_location[f] = os.path.join(folder_path, f)

else:
  print(f"Folder not found: {folder_path}")

print(file_location)

Mounted at /content/drive
{'seniority_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_test_set.csv', 'unlabelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/unlabelled_development_set.csv', 'salary_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_development_set.csv', 'seniority_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_development_set.csv', 'work_arrangements_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_test_set.csv', 'salary_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_test_set.csv', 'work_arrangements_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_development_set.csv', 'results': '/content/drive/MyDrive/AB_job_data_files/results', 'wandb': '/content/drive/MyDrive/AB_job_data_files/wandb', 't5-small-work-arrangement-finetuned': '/content/d

Get all the y labels from development set

In [ ]:
# DEV set
import pandas as pd

true_label = pd.read_csv(file_location['seniority_labelled_development_set.csv'])['y_true']
set(true_label)

{'1st year apprentice',
 '2nd year apprentice',
 '4th year apprentice',
 'advanced',
 'apprentice',
 'apprentice-first-year',
 'assistant',
 'assistant director',
 'assistant head',
 'assistant manager',
 'associate',
 'associate director',
 'board',
 'chief',
 'coordinator',
 'deputy',
 'director',
 'entry level',
 'entry level assistant',
 'entry-level',
 'entry-level to intermediate',
 'executive',
 'experienced',
 'experienced assistant',
 'general-manager',
 'graduate',
 'graduate/junior',
 'head',
 'independent',
 'intermediate',
 'intermediate apprentice',
 'intermediate to senior',
 'junior',
 'junior assistant',
 'junior-intermediate',
 'lead',
 'level 2',
 'manager',
 'mid-level',
 'mid-senior',
 'middle management',
 'middle-management',
 'owner',
 'owner-operator',
 'post-doctoral',
 'postdoctoral',
 'principal',
 'qualified',
 'regional head',
 'second-in-command',
 'senior',
 'senior assistant',
 'senior associate',
 'senior executive',
 'senior head',
 'senior lead',
 's

Create a function to extract seniority from the Test set

In [ ]:
import re
import random

# Your exact labels
seniority_labels = set(true_label)

def preprocess(text):
    """Lowercase and remove unnecessary punctuation (basic)."""
    return re.sub(r'[^\w\s/-]', '', text.lower())

def classify_seniority(job_ad_text):
    text = preprocess(job_ad_text)

    # Count matches
    label_counts = {
        label: len(re.findall(rf'\b{re.escape(label.lower())}\b', text))
        for label in seniority_labels
    }

    max_count = max(label_counts.values())

    # Get all labels with the max count
    top_labels = [label for label, count in label_counts.items() if count == max_count]

    # Choose randomly among top matches (even if max_count is 0)
    return random.choice(top_labels)


Sample example

In [ ]:
ad_text = "This is an exciting opportunity for a graduate/junior developer to join our team."
print(classify_seniority(ad_text))  # Likely output: 'graduate/junior'


graduate


Test set for testing the function

In [ ]:
# TEST set
import pandas as pd

seniority_test = pd.read_csv(file_location['seniority_labelled_test_set.csv'])

results = 0
count = 0
for index, row in seniority_test.iterrows(): # Iterate through the rows of the dataframe
    predicted_label = classify_seniority(row['job_ad_details']).lower().strip()
    true_label =  row['y_true'].lower().strip()

    print(predicted_label ,'---', true_label)
    print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


manager --- senior
False
manager --- experienced
False
student --- entry level
False
experienced --- senior
False
experienced --- intermediate
False
senior head --- experienced
False
middle management --- experienced
False
manager --- entry level
False
manager --- entry level
False
head --- experienced
False
experienced --- senior
False
independent --- experienced
False
manager --- experienced
False
apprentice-first-year --- senior
False
trainee --- trainee
True
experienced --- experienced
True
manager --- experienced
False
sous --- intermediate
False
apprentice --- apprentice
True
intermediate apprentice --- entry level
False
apprentice-first-year --- entry level
False
manager --- entry level
False
specialist --- experienced
False
experienced --- experienced
True
experienced --- experienced
True
experienced --- experienced
True
specialist --- graduate
False
manager --- senior
False
standard --- senior
False
mid-level --- intermediate
False
specialist --- head
False
entry-level --- sen